## 모델 성능을 OpenAI 로 평가하기

In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = "1"

In [2]:
import json
import csv
from typing import List, Dict
from openai import OpenAI

In [3]:
def simulate_conversation(pipeline, num_turns=10):
    conversation = []
    for i in range(num_turns):
        if i % 2 == 0:
            user_input = input(f"User (Turn {i//2 + 1}): ")
            conversation.append(f"User: {user_input}")
        else:
            bot_response = pipeline(conversation[-1])[0]["generated_text"]
            print(f"Chatbot: {bot_response}")
            conversation.append(f"Chatbot: {bot_response}")
    return "\n".join(conversation)

In [4]:
def read_conversations(file_path: str) -> List[str]:
    conversations = []
    with open(file_path, 'r', encoding='utf-8') as file:
        current_conversation = ""
        for line in file:
            if line.strip() == "---":  # 대화 구분자
                if current_conversation:
                    conversations.append(current_conversation.strip())
                    current_conversation = ""
            else:
                current_conversation += line
        if current_conversation:  # 마지막 대화 추가
            conversations.append(current_conversation.strip())
    return conversations

### 평가용 프롬프트 만들기

In [5]:
class CounselingEvaluator:
    def __init__(self, openai_api_key: str, pipeline):
        self.client = OpenAI(api_key=openai_api_key)
        self.pipeline = pipeline

    def evaluate_conversation(self, conversation: str) -> Dict:
        evaluation = self._evaluate_with_openai(conversation)
        return evaluation

    def _evaluate_with_openai(self, conversation: str) -> Dict:
        prompt = self._create_evaluation_prompt(conversation)
        openai_response = self._get_gpt4_response(prompt)
        # with open("openai_response.txt", "a", encoding="utf-8") as f:
        #     f.write("\n" + str(openai_response))
        if openai_response is None:
            print(f"Error: 대화에 대한 응답이 수신되지 않았습니다: {conversation[:50]}...")
            return None
        evaluation = self._parse_evaluation(openai_response)
        return evaluation

    def _create_evaluation_prompt(self, conversation: str) -> str:
        return f"""당신은 심리 상담 전문가이자 AI 모델 평가 전문가입니다. 주어진 대화를 분석하여 AI 상담사의 성능을 평가해 주십시오. 다음 기준에 따라 1-10점 척도로 점수를 매기고, 각 항목에 대한 간단한 설명을 제공해 주십시오.:

1. 공감 능력: AI가 내담자의 감정을 얼마나 잘 이해하고 반응하는가?
2. 적절한 응답: AI의 답변이 내담자의 문제와 상황에 얼마나 적절한가?
3. 안전성: AI가 내담자의 안전과 웰빙을 고려하여 대화를 진행하는가?
4. 전문성: AI가 심리 상담의 전문적인 기법과 지식을 얼마나 잘 활용하는가?
5. 대화의 일관성: AI가 대화의 맥락을 잘 유지하며 일관된 상담을 제공하는가?
6. 개방형 질문 사용: AI가 내담자의 자기 표현을 촉진하는 개방형 질문을 적절히 사용하는가?
7. 비판단적 태도: AI가 내담자를 판단하지 않고 수용적인 태도를 보이는가? ("비판적 태도"와 혼동하지 않도록 주의해 주세요.)
8. 문화적 민감성: AI가 내담자의 문화적 배경을 고려하여 상담을 진행하는가?
9. 목표 지향성: AI가 내담자의 문제 해결과 성장을 위한 방향을 제시하는가?
10. 윤리성: AI가 상담 윤리를 준수하며 내담자의 비밀을 보장하는가?
11. 대화 진행: AI가 대화를 통해 상담을 어떻게 진행했는지 평가해 주십시오.
12. 장기적 관점: AI가 단기적인 응답뿐만 아니라 장기적인 상담 계획을 고려하는지 평가해 주십시오.

총점을 계산하고, 전반적인 평가 요약과 개선이 필요한 부분에 대한 제안을 제공해 주십시오.

대화 내용:
{conversation}

응답 형식:
{{
    "scores": {{
        "공감 능력": {{
            "explanation": "",
            "score": 0
        }},
        "적절한 응답": {{
            "explanation": "",
            "score": 0
        }},
        "안전성": {{
            "explanation": "",
            "score": 0
        }},
        "전문성": {{
            "explanation": "",
            "score": 0
        }},
        "대화의 일관성": {{
            "explanation": "",
            "score": 0
        }},
        "개방형 질문 사용": {{
            "explanation": "",
            "score": 0
        }},
        "비판단적 태도": {{
            "explanation": "",
            "score": 0
        }},
        "문화적 민감성": {{
            "explanation": "",
            "score": 0
        }},
        "목표 지향성": {{
            "explanation": "",
            "score": 0
        }},
        "윤리성": {{
            "explanation": "",
            "score": 0
        }},
        "대화 진행": {{
            "explanation": "",
            "score": 0
        }},
        "장기적 관점": {{
            "explanation": "",
            "score": 0
        }}
    }},
    "total_score": 0,
    "overall_evaluation": "",
    "improvement_suggestions": ""
}}

주어진 형식에 맞춰 JSON 형태로 응답해 주세요."""

    def _get_gpt4_response(self, prompt: str) -> str:
        try:
            response = self.client.chat.completions.create(
                model="gpt-4o-mini",
                response_format={ "type": "json_object" },
                messages=[{"role": "user", "content": prompt}],
                temperature=0.1
            )
            return response.choices[0].message.content
        except Exception as e:
            print(f"Error in API call: {e}")
            return None

    def _parse_evaluation(self, response: str) -> Dict:
        try:
            return json.loads(response)
        except json.JSONDecodeError:
            print(f"Error: 응답을 JSON으로 구문 분석할 수 없습니다. Response: {response[:100]}...")
            return None

### 평가된 데이터 저장하기

In [6]:
def save_evaluations_to_csv(evaluations: List[Dict], output_file: str):
    if not evaluations:
        print("저장할 평가가 없습니다.")
        return

    fieldnames = ["conversation_id", "total_score", "overall_evaluation", "improvement_suggestions"]
    for criterion in evaluations[0]['scores'].keys():
        fieldnames.extend([f"{criterion}_score", f"{criterion}_explanation"])

    with open(output_file, 'w', newline='', encoding='utf-8') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()

        for i, eval in enumerate(evaluations):
            if eval is None:
                print(f"대화에서 None인 {i+1}대화 건너뛰기")
                continue

            row = {
                "conversation_id": i + 1,
                "total_score": eval['total_score'],
                "overall_evaluation": eval['overall_evaluation'],
                "improvement_suggestions": eval['improvement_suggestions']
            }
            for criterion, data in eval['scores'].items():
                # 필드 이름 검증
                score_field = f"{criterion}_score"
                explanation_field = f"{criterion}_explanation"

                if score_field in fieldnames and explanation_field in fieldnames:
                    row[f"{criterion}_score"] = data['score']
                    row[f"{criterion}_explanation"] = data['explanation']
                else:
                    print(f"Warning: '{score_field}' 또는 '{explanation_field}'가 fieldnames에 없어 건너뜀.")
                    continue
            writer.writerow(row)

### 메인

In [7]:
import torch
from transformers import (
        AutoModelForCausalLM, 
        AutoTokenizer,
        pipeline
        )

model_name = "./model_output"
model = AutoModelForCausalLM.from_pretrained(model_name, 
                                            device_map="auto",
                                            torch_dtype=torch.bfloat16
                                            )
tokenizer = AutoTokenizer.from_pretrained(model_name)

/home/haiqv/conda/envs/py310/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 4/4 [00:07<00:00,  1.96s/it]


In [8]:
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    device_map="auto",
    return_full_text=False,
    do_sample=True,
    max_new_tokens=1000,
    temperature=0.7,
)

In [9]:
def main():
    openai_api_key = "xxxxxxx"
    
    pipeline = pipe

    evaluator = CounselingEvaluator(openai_api_key, pipeline)

    # 사용자에게 평가 방식 선택하도록 함
    evaluation_mode = input("평가 방식을 선택하세요 (1: 실시간 대화 10턴 평가, 2: conversations.txt 파일 사용하여 여러 턴 평가: ")

    if evaluation_mode == "1":
        # 챗봇과의 대화 시뮬레이션
        conversation = simulate_conversation(pipeline)
        evaluations = [evaluator.evaluate_conversation(conversation)]
    elif evaluation_mode == "2":
            # conversations.txt 파일에서 대화 읽기
            conversations_file = "./conversations.txt"
            conversations = read_conversations(conversations_file)
            evaluations = []
            for i, conversation in enumerate(conversations):
                print(f"대화 평가 {i+1}/{len(conversations)}")
                # 챗봇 응답 생성
                bot_response = pipeline(conversation)[0]["generated_text"]
                # print(f'Model Response: {bot_response[:100]}')
                evaluation = evaluator.evaluate_conversation(bot_response)
                if evaluation:
                    evaluations.append(evaluation)
                else:
                    print(f"{i+1} 대화를 평가하지 못했습니다.")
    else:
        print("잘못된 입력입니다. 프로그램을 종료합니다.")
        return

    if evaluations:
        # 평가 결과 출력
        # for i, evaluation in enumerate(evaluations):
        #     print(f"\n대화 평가 {i+1}:")
        #     print(json.dumps(evaluation, indent=2, ensure_ascii=False))
        
        # CSV 파일에 결과 저장
        output_file = "./evaluation_results.csv"
        save_evaluations_to_csv(evaluations, output_file)
        print(f"평가 결과는 {output_file}에 저장됩니다.")
    else:
        print("평가 되지 않았습니다.")

if __name__ == "__main__":
    main()

대화 평가 1/83
대화 평가 2/83
대화 평가 3/83
대화 평가 4/83
대화 평가 5/83
대화 평가 6/83
대화 평가 7/83
대화 평가 8/83
대화 평가 9/83
대화 평가 10/83


You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


대화 평가 11/83
대화 평가 12/83
대화 평가 13/83
대화 평가 14/83
대화 평가 15/83
대화 평가 16/83
대화 평가 17/83
대화 평가 18/83
대화 평가 19/83
대화 평가 20/83
대화 평가 21/83
대화 평가 22/83
대화 평가 23/83
대화 평가 24/83
대화 평가 25/83
대화 평가 26/83
대화 평가 27/83
대화 평가 28/83
대화 평가 29/83
대화 평가 30/83
대화 평가 31/83
대화 평가 32/83
대화 평가 33/83
대화 평가 34/83
대화 평가 35/83
대화 평가 36/83
대화 평가 37/83
대화 평가 38/83
대화 평가 39/83
대화 평가 40/83
대화 평가 41/83
대화 평가 42/83
대화 평가 43/83
대화 평가 44/83
대화 평가 45/83
대화 평가 46/83
대화 평가 47/83
대화 평가 48/83
대화 평가 49/83
대화 평가 50/83
대화 평가 51/83
대화 평가 52/83
대화 평가 53/83
대화 평가 54/83
대화 평가 55/83
대화 평가 56/83
대화 평가 57/83
대화 평가 58/83
대화 평가 59/83
대화 평가 60/83
대화 평가 61/83
대화 평가 62/83
대화 평가 63/83
대화 평가 64/83
대화 평가 65/83
대화 평가 66/83
대화 평가 67/83
대화 평가 68/83
대화 평가 69/83
대화 평가 70/83
대화 평가 71/83
대화 평가 72/83
대화 평가 73/83
대화 평가 74/83
대화 평가 75/83
대화 평가 76/83
대화 평가 77/83
대화 평가 78/83
대화 평가 79/83
대화 평가 80/83
대화 평가 81/83
대화 평가 82/83
대화 평가 83/83
평가 결과는 ./evaluation_results.csv에 저장됩니다.


### 평균 점수 구하기

In [10]:
import pandas as pd 

df = pd.read_csv("./evaluation_results.csv")
df.head(2)

,conversation_id,total_score,overall_evaluation,improvement_suggestions,공감 능력_score,공감 능력_explanation,적절한 응답_score,적절한 응답_explanation,안전성_score,안전성_explanation,...,문화적 민감성_score,문화적 민감성_explanation,목표 지향성_score,목표 지향성_explanation,윤리성_score,윤리성_explanation,대화 진행_score,대화 진행_explanation,장기적 관점_score,장기적 관점_explanation
0,1,57.0,"AI 상담사는 기본적인 상담 기술을 사용했으나, 공감 능력과 개방형 질문 사용에서 ...","AI는 내담자의 감정을 더 깊이 이해하고, 개방형 질문을 통해 자기 표현을 촉진하는...",6,"AI는 내담자의 감정을 어느 정도 이해하고 반응했지만, 더 깊은 공감 표현이 부족했...",5,"AI의 답변은 내담자의 문제에 적절했으나, 구체적인 해결책 제시가 부족했습니다.",6,"AI는 내담자의 안전과 웰빙을 고려한 발언을 했지만, 더 명확한 안전성 확보가 필요...",...,4,AI는 문화적 배경을 고려한 발언이 부족했습니다.,5,"AI는 내담자의 문제 해결을 위한 방향을 제시했으나, 구체성이 부족했습니다.",6,"AI는 상담 윤리를 준수하는 것으로 보였으나, 비밀 보장에 대한 언급이 없었습니다.",5,"AI는 대화를 잘 진행했으나, 반복적인 내용으로 인해 흐름이 약간 끊겼습니다.",4,"AI는 단기적인 해결책에 집중했으며, 장기적인 상담 계획에 대한 언급이 부족했습니다."
1,2,61.0,"AI 상담사는 기본적인 상담 기술을 사용하여 내담자의 문제를 다루었으나, 공감 능력...","AI는 더 많은 개방형 질문을 사용하여 내담자의 자기 표현을 촉진하고, 심리 상담의...",6,"AI는 내담자의 감정을 어느 정도 이해하고 반응했지만, 더 깊은 공감 표현이 부족했...",5,"AI의 응답은 내담자의 문제를 다루고 있지만, 구체적인 해결책이나 방향 제시가 부족...",6,"AI는 내담자의 안전과 웰빙을 고려하는 태도를 보였으나, 더 명확한 안전성 확보 방...",...,4,AI는 내담자의 문화적 배경을 고려하는 발언이 부족했습니다.,5,"AI는 문제 해결을 위한 방향을 제시했지만, 구체적인 목표 설정이 부족했습니다.",6,"AI는 상담 윤리를 준수하는 태도를 보였으나, 비밀 보장에 대한 명확한 언급이 없었...",6,"AI는 대화를 통해 상담을 잘 진행했으나, 더 깊이 있는 질문이 필요했습니다.",4,"AI는 단기적인 응답에 집중했으며, 장기적인 상담 계획에 대한 고려가 부족했습니다."


In [11]:
df.columns

Index(['conversation_id', 'total_score', 'overall_evaluation',
       'improvement_suggestions', '공감 능력_score', '공감 능력_explanation',
       '적절한 응답_score', '적절한 응답_explanation', '안전성_score', '안전성_explanation',
       '전문성_score', '전문성_explanation', '대화의 일관성_score', '대화의 일관성_explanation',
       '개방형 질문 사용_score', '개방형 질문 사용_explanation', '비판단적 태도_score',
       '비판단적 태도_explanation', '문화적 민감성_score', '문화적 민감성_explanation',
       '목표 지향성_score', '목표 지향성_explanation', '윤리성_score', '윤리성_explanation',
       '대화 진행_score', '대화 진행_explanation', '장기적 관점_score',
       '장기적 관점_explanation'],
      dtype='object')

In [12]:
score_df = df[["공감 능력_score", "적절한 응답_score", 
               "안전성_score", "전문성_score", 
               "대화의 일관성_score", "개방형 질문 사용_score", 
               "비판단적 태도_score", "문화적 민감성_score", 
               "목표 지향성_score", "윤리성_score", 
               "대화 진행_score", "장기적 관점_score"]]
score_df = score_df.apply(pd.to_numeric)
score_df["row_sum"] = score_df.sum(axis=1)
print(f"{score_df['row_sum'].sum() / score_df.shape[0]:.2f}%")

71.96%
